# Higher Ed Python Code

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import datetime
import xlsxwriter
import re

#### Importing Data

In [2]:
# Importing all of the sheets from the UiPath output and creating a variable for each
# Appending all variables into a list
HigherEdSheets = pd.ExcelFile("HigherEdOutput/HigherEd.xlsx").sheet_names
List_of_Schools = []
for i in HigherEdSheets:
    tab_name = i.replace(" ","_")
    print(i)
    globals()[tab_name] = pd.DataFrame(pd.read_excel("HigherEdOutput/HigherEd.xlsx", sheet_name = i))
    globals()[tab_name].name = i
    List_of_Schools.append(globals()[tab_name])

University of Florida
Miami Dade College


#### Cleaning Up Data

In [20]:
opp_dict = {'School':[],'Bid_Number':[],'Bid_Start_Date':[], 'Bid_Close_Date':[],"Bid_Description":[], "Hyperlink":[]}
Opportunities = pd.DataFrame(opp_dict)

def crop_columns(df1,df2):
    # df1 should be 'Opportunities' dataframe
    # df2 should be the dataframe in question
    cropped_list = []
    for i in df2.columns:
        if i in df1.columns:
            pass
        else:
            cropped_list.append(i)
    return cropped_list


In [204]:
Opportunities

,Bid_Number,Bid_Start_Date,Bid_Close_Date,Bid_Description,Hyperlink


### University of Florida

In [199]:
# University_of_Florida.columns
# University_of_Florida.rename(columns = {'Bid Number':'Bid_Number','Bid Opening Date':'Bid_Start_Date'},inplace = True)
# University_of_Florida.drop(columns = crop_columns(Opportunities,University_of_Florida), inplace = True)

# # Clean up the dates to just grab the first one and convert it to a datetime
# University_of_Florida.Bid_Start_Date = pd.to_datetime([University_of_Florida.Bid_Start_Date[i].split(' at')[0] for i in range(len(University_of_Florida.Bid_Start_Date))])


# Opportunities.append(University_of_Florida)

# Table Scraping

In [39]:
List_Schools = pd.read_excel("List_of_Schools.xlsx", sheet_name='Schools')

In [40]:
List_Schools

,School,URL
0,University of Central Florida,https://procurement.ucf.edu/solicitations/
1,University of South Florida,https://bids.sciquest.com/apps/Router/PublicEv...
2,Univeristy of North Florida,https://bids.sciquest.com/apps/Router/PublicEv...
3,Florida State University,https://bids.sciquest.com/apps/Router/PublicEv...
4,University of Florida,https://procurement.ufl.edu/vendors/schedule-o...
5,Miami Dade College,https://www.bidnetdirect.com/florida/miamidade...
6,Florida International Univeristy,https://bids.sciquest.com/apps/Router/PublicEv...


### For sciquest sites

In [49]:
def clean_sciquest(school, url): # Takes tables in the sciquest format and cleans them up
    df = pd.read_html(url)
    def find_between(string, text1, text2): # finds the information between two strings
        result=(re.search(text1+'(.*)'+text2, string))
        final_text = result.group(1)
        return final_text.strip()
    def clean_date(string, text1, text2): # turns a string date into a datetime using the find_between function
        date_clean_1 = find_between(string,text1,text2).split(" ")[0:3]
        date_clean_2 = date_clean_1[0]+" "+date_clean_1[1]+" "+date_clean_1[2]
        return date_clean_2
    
#     clean_dict = {'Bid_Number':[],'Bid_Start_Date':[], 'Bid_Close_Date':[],"Bid_Description":[], 'Point of Contact':[]}
    clean_dict = {'Bid_Number':[],'Bid_Start_Date':[], 'Bid_Close_Date':[],"Bid_Description":[], 'Hyperlink':[]}
    clean_df = pd.DataFrame(clean_dict)
    
    for i in range(len(df[0].Details)):
        clean_df.loc[i,'School'] = school
        clean_df.loc[i,'Bid_Number'] = find_between(df[0].Details[i],"Number",'Contact')
        clean_df.loc[i,'Bid_Start_Date'] = pd.to_datetime(clean_date(df[0].Details[i],'Open','Close'))
        clean_df.loc[i,'Bid_Close_Date'] = pd.to_datetime(clean_date(df[0].Details[i],'Close','Type'))
        clean_df.loc[i,'Bid_Description'] = df[0].Details[i].split("Open")[0]
        #clean_df.loc[i,'Point of Contact'] = find_between(df[0].Details[i],"Contact",'Details')
    return clean_df

In [50]:
opp_dict = {'School':[],'Bid_Number':[],'Bid_Start_Date':[], 'Bid_Close_Date':[],"Bid_Description":[], "Hyperlink":[]}
Opportunities = pd.DataFrame(opp_dict)

In [51]:
for i,j in zip(List_Schools['School'],List_Schools['URL']):
    if 'sciquest' in j:
        Opportunities = Opportunities.append(clean_sciquest(i,j))
Opportunities.reset_index(inplace= True, drop = True)

In [52]:
Opportunities

,School,Bid_Number,Bid_Start_Date,Bid_Close_Date,Bid_Description,Hyperlink
0,University of South Florida,2021-030-ITN-PRO,2021-01-27 14:01:00,2021-02-25 15:00:00,Owner-Controlled Insurance Program for Constru...,NaN
1,Univeristy of North Florida,GC-2101-01,2021-02-05 17:00:00,2021-03-02 14:00:00,Building 45 Fume Hood Replacement The Univers...,NaN
2,Florida State University,ITN 6223-6,2021-02-15 08:00:00,2021-03-12 15:00:00,FSU Remote Virtual Campus and Self-Guided Camp...,NaN
3,Florida State University,ITN 6221-4,2021-02-11 16:30:00,2021-03-11 15:00:00,Enterprise File and Object Storage Solution F...,NaN
4,Florida State University,ITN 6219-A,2021-02-15 00:00:00,2021-03-09 15:00:00,FSU Facilities Uniforms with Related Product a...,NaN
5,Florida State University,ITN 6198-4,2021-01-26 15:00:00,2021-03-02 15:00:00,Oracle 3rd Party Support Florida State Univer...,NaN
6,Florida State University,ITN 6215-A,2021-01-22 00:00:00,2021-02-25 15:00:00,FSU Oglesby Student Union Partitions Responde...,NaN
7,Florida State University,ITB 6220-A,2021-02-03 00:00:00,2021-02-23 16:30:00,Carnaghi Arts Building Re-Roofing Respondent ...,NaN
8,Florida State University,ITB 6218-A,2021-01-26 00:00:00,2021-02-23 15:00:00,FSU UCA- South Center Roof Tile Repair a Resp...,NaN
9,Florida International Univeristy,ITN-2021-00051,2021-02-16 17:00:00,2021-02-26 14:00:00,Consulting and Design Services: Federal Resear...,NaN


In [48]:
Opportunities.to_excel('Opportunities.xlsx', sheet_name='Main Page', index=False)

In [ ]:
# def relevance_check(df): # drops irrelevance opporutnities
#     for i in df.Bid_Description:
        

In [54]:
# df = []
# df_list = []
# for i in range(1, 3):
#     url_head = 'https://bids.sciquest.com/apps/Router/PublicEvent?tab=PHX_NAV_SourcingAllOpps&CustomerOrg=USFlorida&tmstmp=1613600246398.html?pageNum=%d' %i
#     df_list.append(pd.read_html(url)[0])

# df = pd.concat(df_list)

In [2]:
# from html.parser import HTMLParser
# from html.entities import name2codepoint

# class MyHTMLParser(HTMLParser):
#     def handle_starttag(self, tag, attrs):
#         print("Start tag:", tag)
#         for attr in attrs:
#             print("     attr:", attr)

#     def handle_endtag(self, tag):
#         print("End tag  :", tag)

#     def handle_data(self, data):
#         print("Data     :", data)

#     def handle_comment(self, data):
#         print("Comment  :", data)

#     def handle_entityref(self, name):
#         c = chr(name2codepoint[name])
#         print("Named ent:", c)

#     def handle_charref(self, name):
#         if name.startswith('x'):
#             c = chr(int(name[1:], 16))
#         else:
#             c = chr(int(name))
#         print("Num ent  :", c)

#     def handle_decl(self, data):
#         print("Decl     :", data)

# parser = MyHTMLParser()